# Simulation Results

In [169]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import copy
from matplotlib.backends.backend_pdf import PdfPages

# Do not limit number of rows displayed
pd.set_option('display.max_rows', None)
# constants
TO = 295.0
# benchmark_categories = list(['wolf/18D', 'wolf/19A', 'wolf/19B', 'wolf/19C', 'beem', 'goel', '19/mann', '20/mann'])
array_categories = list(['wolf/18A', 'wolf/19A', 'wolf/19B', 'wolf/19C', '19/mann', '20/mann'])
sims = "lpar/sims-ncnb"
resolve = "lpar/resolve-ncnb"

In [170]:
def clean_b2ir_csv(in_file, btor=False):
  df = pd.read_csv(in_file)
  df = df.rename(columns = {'index': 'base'})
  return df

def clean_btorsim_csv(in_file, btor=False):
  df = pd.read_csv(in_file)
  df = df.rename(columns = {'index': 'base'})
  return df

def clean_seahorn_csv(in_file):
  df = pd.read_csv(in_file)
  df = df.rename(columns = {'index': 'base', 'status': 'sea_status'})
  # set trivial unsat
  df.loc[df['base'] == 'simple_alu.btor', 'sea_status'] = 'unsat'
  df.loc[df['base'] == 'vcegar_QF_BV_itc99_b13_p10.btor2', 'sea_status'] = 'unsat'
  if 'sea_status' in df.columns:
    # identify timeouts
    df['sea_status'] = df['sea_status'].fillna('unknown')
    df['sea_status'] = df['sea_status'].astype(str)
    df.loc[df['sea_status'] == 'unknown', 'seahorn_total'] = 1000
    return df[['base', 'seahorn_total', 'sea_status']]
  return df


## Collect CSVs

### BtorSim


In [171]:
def getDataBtorSim(sims):
  BTORSIM_A_18A_WOLF = "/home/jetafese/hwmc20-mlir/runbench/"+ sims + "/btorsim_array_18A.csv"
  BTORSIM_A_19A_WOLF = "/home/jetafese/hwmc20-mlir/runbench/"+ sims + "/btorsim_array_19A.csv"
  BTORSIM_A_19B_WOLF = "/home/jetafese/hwmc20-mlir/runbench/"+ sims + "/btorsim_array_19B.csv"
  BTORSIM_A_19C_WOLF = "/home/jetafese/hwmc20-mlir/runbench/"+ sims + "/btorsim_array_19C.csv"
  BTORSIM_A_19_MANN = "/home/jetafese/hwmc20-mlir/runbench/"+ sims + "/btorsim_array_19_mann.csv"
  BTORSIM_A_20_MANN = "/home/jetafese/hwmc20-mlir/runbench/"+ sims + "/btorsim_array_20_mann.csv"

  bA20_18A = clean_btorsim_csv(BTORSIM_A_18A_WOLF)
  bA20_19A = clean_btorsim_csv(BTORSIM_A_19A_WOLF)
  bA20_19B = clean_btorsim_csv(BTORSIM_A_19B_WOLF)
  bA20_19C = clean_btorsim_csv(BTORSIM_A_19C_WOLF)
  bA20_19mann = clean_btorsim_csv(BTORSIM_A_19_MANN)
  bA20_20mann = clean_btorsim_csv(BTORSIM_A_20_MANN)
  btorsim_array_results = list([bA20_18A, bA20_19A, bA20_19B, bA20_19C, bA20_19mann, bA20_20mann])
  return btorsim_array_results

### Btor2MLIR 


In [172]:
def getDataBtor2MLIR(runfolder):
  BTOR2MLIR_A_18A_WOLF = "/home/jetafese/hwmc20-mlir/runbench/"+ runfolder + "/btor2mlir_array_18A.csv"
  BTOR2MLIR_A_19A_WOLF = "/home/jetafese/hwmc20-mlir/runbench/"+ runfolder + "/btor2mlir_array_19A.csv"
  BTOR2MLIR_A_19B_WOLF = "/home/jetafese/hwmc20-mlir/runbench/"+ runfolder + "/btor2mlir_array_19B.csv"
  BTOR2MLIR_A_19C_WOLF = "/home/jetafese/hwmc20-mlir/runbench/"+ runfolder + "/btor2mlir_array_19C.csv"
  BTOR2MLIR_A_19_MANN = "/home/jetafese/hwmc20-mlir/runbench/"+ runfolder + "/btor2mlir_array_19_mann.csv"
  BTOR2MLIR_A_20_MANN = "/home/jetafese/hwmc20-mlir/runbench/"+ runfolder + "/btor2mlir_array_20_mann.csv"

  birA20_18A = clean_b2ir_csv(BTOR2MLIR_A_18A_WOLF)
  birA20_19A = clean_b2ir_csv(BTOR2MLIR_A_19A_WOLF)
  birA20_19B = clean_b2ir_csv(BTOR2MLIR_A_19B_WOLF)
  birA20_19C = clean_b2ir_csv(BTOR2MLIR_A_19C_WOLF)
  birA20_19mann = clean_b2ir_csv(BTOR2MLIR_A_19_MANN)
  birA20_20mann = clean_b2ir_csv(BTOR2MLIR_A_20_MANN)
  btor2mlir_array_results = list([birA20_18A, birA20_19A, birA20_19B, birA20_19C, birA20_19mann, birA20_20mann])
  return btor2mlir_array_results

## Compiled Results

In [173]:
def getResultsFor(btorsimRun, btor2mlirRun):
  concat_dfs = list()
  for (btorsim, b2ir) in zip(getDataBtorSim(btorsimRun), getDataBtor2MLIR(btor2mlirRun)):
    b2ir = b2ir.rename(columns = {'cycles': 'b2ir_cycles'})
    btorsim = btorsim.rename(columns = {'cycles': 'btor_cycles'})
    df = pd.merge(btorsim, b2ir[['base', 'b2ir_cycles']], how='inner', on='base')
    # print("btor_cycles: ", round(df.loc[:, 'btor_cycles'].mean()))
    # print("b2ir_cycles: ", round(df.loc[:, 'b2ir_cycles'].mean()))
    concat_dfs.append(df)
  return concat_dfs

In [191]:
def printTotalStats(df, final, bench):
  # BtorSim
  minBtorSim = round(df.loc[:, 'btor_cycles'].min())
  meanBtorSim = round(df.loc[:, 'btor_cycles'].mean())
  maxBtorSim = round(df.loc[:, 'btor_cycles'].max())
  # B2IR
  minB2IR = round(df.loc[:, 'b2ir_cycles'].min())
  meanB2IR = round(df.loc[:, 'b2ir_cycles'].mean())
  maxB2IR = round(df.loc[:, 'b2ir_cycles'].max())
  # Speedup B2IR
  speedupB2IR = round(meanB2IR/meanBtorSim)

  entry = pd.DataFrame.from_dict({
    "base": [bench],
    # BtorSim
    "btorsim_min":  [minBtorSim],
    "btorsim_mean":  [meanBtorSim],
    "btorsim_max":  [maxBtorSim],
    # B2IR
    "b2ir_min":  [minB2IR],
    "b2ir_mean":  [meanB2IR],
    "b2ir_max":  [maxB2IR],
    # speedup
    "speedup" : [speedupB2IR]
  })

  return pd.concat([final, entry], ignore_index=True)

In [192]:
reportDF = pd.DataFrame(columns=['base', 'btorsim_min', 'btorsim_mean', 'btorsim_max',
                                 'b2ir_min', 'b2ir_mean', 'b2ir_max', 'speedup'])
for (bDF, category) in zip(getResultsFor("lpar/sims-nb-2", "lpar/resolve-nb-2"), array_categories):
  reportDF = printTotalStats (bDF, reportDF, category)

btor_cycles:  22
b2ir_cycles:  4608
btor_cycles:  396
b2ir_cycles:  0
btor_cycles:  1
b2ir_cycles:  28772
btor_cycles:  2
b2ir_cycles:  0
btor_cycles:  1
b2ir_cycles:  0
btor_cycles:  1738
b2ir_cycles:  74316


In [184]:
display(reportDF)
# display(reportDF[['base', 'btorsim_min', 'btorsim_mean', 'btorsim_max', 'b2ir_min', 'b2ir_mean', 'b2ir_max']].to_latex())
display(reportDF[['base', 'btorsim_mean', 'b2ir_mean', 'speedup']].to_latex())

,base,btorsim_min,btorsim_mean,btorsim_max,b2ir_min,b2ir_mean,b2ir_max,speedup
0,wolf/18A,1,22,411,0,4608,92146,209
1,wolf/19A,255,396,500,0,0,0,0
2,wolf/19B,1,1,1,27847,28772,29332,28772
3,wolf/19C,1,2,4,0,0,1,0
4,19/mann,1,1,1,0,0,0,0
5,20/mann,1,1738,4960,0,74316,117433,43


'\\begin{tabular}{lllll}\n\\toprule\n & base & btorsim_mean & b2ir_mean & speedup \\\\\n\\midrule\n0 & wolf/18A & 22 & 4608 & 209 \\\\\n1 & wolf/19A & 396 & 0 & 0 \\\\\n2 & wolf/19B & 1 & 28772 & 28772 \\\\\n3 & wolf/19C & 2 & 0 & 0 \\\\\n4 & 19/mann & 1 & 0 & 0 \\\\\n5 & 20/mann & 1738 & 74316 & 43 \\\\\n\\bottomrule\n\\end{tabular}\n'